This is a `bqplot` recreation of Mike Bostock's [Wealth of Nations](https://bost.ocks.org/mike/nations/). This was also done by [Gapminder](http://www.gapminder.org/world/#$majorMode=chart$is;shi=t;ly=2003;lb=f;il=t;fs=11;al=30;stl=t;st=t;nsl=t;se=t$wst;tts=C$ts;sp=5.59290322580644;ti=2013$zpv;v=0$inc_x;mmid=XCOORDS;iid=phAwcNAVuyj1jiMAkmq1iMg;by=ind$inc_y;mmid=YCOORDS;iid=phAwcNAVuyj2tPLxKvvnNPA;by=ind$inc_s;uniValue=8.21;iid=phAwcNAVuyj0XOoBL_n5tAQ;by=ind$inc_c;uniValue=255;gid=CATID0;by=grp$map_x;scale=log;dataMin=194;dataMax=96846$map_y;scale=lin;dataMin=23;dataMax=86$map_s;sma=49;smi=2.65$cd;bd=0$inds=;modified=60). It is originally based on a TED Talk by [Hans Rosling](http://www.ted.com/talks/hans_rosling_shows_the_best_stats_you_ve_ever_seen).

In [1]:
import pandas as pd
import numpy as np
import os

from bqplot import (
    LogScale, LinearScale, OrdinalColorScale, ColorAxis,
    Axis, Scatter, Lines, CATEGORY10, Label, Figure, Tooltip
)

from ipywidgets import HBox, VBox, IntSlider, Play, jslink

In [2]:
initial_year = 1800

#### Cleaning and Formatting JSON Data

In [3]:
data = pd.read_json(os.path.abspath('../data_files/nations.json'))

In [4]:
def clean_data(data):
    for column in ['income', 'lifeExpectancy', 'population']:
        data = data.drop(data[data[column].apply(len) <= 4].index)
    return data

def extrap_interp(data):
    data = np.array(data)
    x_range = np.arange(1800, 2009, 1.)
    y_range = np.interp(x_range, data[:, 0], data[:, 1])
    return y_range

def extrap_data(data):
    for column in ['income', 'lifeExpectancy', 'population']:
        data[column] = data[column].apply(extrap_interp)
    return data

In [5]:
data = clean_data(data)
data = extrap_data(data)

In [6]:
income_min, income_max = np.min(data['income'].apply(np.min)), np.max(data['income'].apply(np.max))
life_exp_min, life_exp_max = np.min(data['lifeExpectancy'].apply(np.min)), np.max(data['lifeExpectancy'].apply(np.max))
pop_min, pop_max = np.min(data['population'].apply(np.min)), np.max(data['population'].apply(np.max))

In [7]:
def get_data(year):
    year_index = year - 1800
    income = data['income'].apply(lambda x: x[year_index])
    life_exp = data['lifeExpectancy'].apply(lambda x: x[year_index])
    pop =  data['population'].apply(lambda x: x[year_index])
    return income, life_exp, pop

#### Creating the Tooltip to display the required fields

`bqplot`'s native `Tooltip` allows us to simply display the data fields we require on a mouse-interaction.

In [8]:
tt = Tooltip(fields=['name', 'x', 'y'], labels=['Country Name', 'Income per Capita', 'Life Expectancy'])

#### Creating the Label to display the year

Staying true to the `d3` recreation of the talk, we place a `Label` widget in the bottom-right of the `Figure` (it inherits the `Figure` co-ordinates when no scale is passed to it). With `enable_move` set to `True`, the `Label` can be dragged around. 

In [9]:
year_label = Label(x=[0.75], y=[0.10], default_size=46, font_weight='bolder', colors=['orange'],
                   text=[str(initial_year)], enable_move=True)

#### Defining Axes and Scales

The inherent skewness of the income data favors the use of a `LogScale`. Also, since the color coding by regions does not follow an ordering, we use the `OrdinalColorScale`.

In [10]:
x_sc = LogScale(min=income_min, max=income_max)
y_sc = LinearScale(min=life_exp_min, max=life_exp_max)
c_sc = OrdinalColorScale(domain=data['region'].unique().tolist(), colors=CATEGORY10[:6])
size_sc = LinearScale(min=pop_min, max=pop_max)

In [11]:
ax_y = Axis(label='Life Expectancy', scale=y_sc, orientation='vertical', side='left', grid_lines='solid')
ax_x = Axis(label='Income per Capita', scale=x_sc, grid_lines='solid')

#### Creating the Scatter Mark with the appropriate size and color parameters passed

To generate the appropriate graph, we need to pass the population of the country to the `size` attribute and its region to the `color` attribute.

In [12]:
# Start with the first year's data
cap_income, life_exp, pop = get_data(initial_year)

In [13]:
wealth_scat = Scatter(x=cap_income, y=life_exp, color=data['region'], size=pop,
                      names=data['name'], display_names=False,
                      scales={'x': x_sc, 'y': y_sc, 'color': c_sc, 'size': size_sc},
                      default_size=4112, tooltip=tt, animate=True, stroke='Black',
                      unhovered_style={'opacity': 0.5})

In [14]:
nation_line = Lines(x=data['income'][0], y=data['lifeExpectancy'][0], colors=['Gray'],
                       scales={'x': x_sc, 'y': y_sc}, visible=False)

#### Creating the Figure

In [15]:
time_interval = 10

In [16]:
fig = Figure(marks=[wealth_scat, year_label, nation_line], axes=[ax_x, ax_y],
             title='Health and Wealth of Nations', animation_duration=time_interval)

#### Using a Slider to allow the user to change the year and a button for animation

Here we see how we can seamlessly integrate `bqplot` into the jupyter widget infrastructure. 

In [17]:
year_slider = IntSlider(min=1800, max=2008, step=1, description='Year', value=initial_year)

When the `hovered_point` of the `Scatter` plot is changed (i.e. when the user hovers over a different element), the entire path of that country is displayed by making the `Lines` object visible and setting it's `x` and `y` attributes.

In [18]:
def hover_changed(change):
    if change.new is not None:
        nation_line.x = data[data['name'] == wealth_scat.names[change.new]]['income'].values[0]
        nation_line.y = data[data['name'] == wealth_scat.names[change.new]]['lifeExpectancy'].values[0]
        nation_line.visible = True
    else:
        nation_line.visible = False
        
wealth_scat.observe(hover_changed, 'hovered_point')

On the slider value `callback` (a function that is triggered everytime the `value` of the slider is changed) we change the `x`, `y` and `size` co-ordinates of the `Scatter`. We also update the `text` of the `Label` to reflect the current year.

In [19]:
def year_changed(change):
    wealth_scat.x, wealth_scat.y, wealth_scat.size = get_data(year_slider.value)
    year_label.text = [str(year_slider.value)]

year_slider.observe(year_changed, 'value')

#### Add an animation button

In [20]:
play_button = Play(min=1800, max=2008, interval=time_interval)
jslink((play_button, 'value'), (year_slider, 'value'))

#### Displaying the GUI

In [21]:
VBox([HBox([play_button, year_slider]), fig])

VBox(children=(HBox(children=(Play(value=1800, interval=10, max=2008, min=1800), IntSlider(value=1800, description='Year', max=2008, min=1800))), Figure(animation_duration=10, axes=[Axis(label='Income per Capita', scale=LogScale(max=119849.28999999999, min=281.91000000000003)), Axis(label='Life Expectancy', orientation='vertical', scale=LinearScale(max=82.870000000000005, min=12.699999999999999), side='left')], fig_margin={'bottom': 60, 'right': 60, 'left': 60, 'top': 60}, layout=Layout(min_width='125px'), marks=[Scatter(color=array(['Sub-Saharan Africa', 'Sub-Saharan Africa', 'Sub-Saharan Africa',
       'Sub-Saharan Africa', 'Sub-Saharan Africa', 'Sub-Saharan Africa',
       'Sub-Saharan Africa', 'Sub-Saharan Africa', 'Sub-Saharan Africa',
       'Sub-Saharan Africa', 'Sub-Saharan Africa', 'Sub-Saharan Africa',
       'Sub-Saharan Africa', 'Sub-Saharan Africa', 'Sub-Saharan Africa',
       'Sub-Saharan Africa', 'Sub-Saharan Africa', 'Sub-Saharan Africa',
       'Sub-Saharan Africa', 'Sub-Saharan Africa', 'Sub-Saharan Africa',
       'Sub-Saharan Africa', 'Sub-Saharan Africa', 'Sub-Saharan Africa',
       'Sub-Saharan Africa', 'Sub-Saharan Africa', 'Sub-Saharan Africa',
       'Sub-Saharan Africa', 'Sub-Saharan Africa', 'Sub-Saharan Africa',
       'Sub-Saharan Africa', 'Sub-Saharan Africa', 'Sub-Saharan Africa',
       'Sub-Saharan Africa', 'Sub-Saharan Africa', 'Sub-Saharan Africa',
       'Sub-Saharan Africa', 'Sub-Saharan Africa', 'Sub-Saharan Africa',
       'Sub-Saharan Africa', 'Sub-Saharan Africa', 'Sub-Saharan Africa',
       'Sub-Saharan Africa', 'Sub-Saharan Africa', 'Sub-Saharan Africa',
       'South Asia', 'South Asia', 'South Asia', 'South Asia',
       'South Asia', 'South Asia', 'South Asia', 'South Asia',
       'Middle East & North Africa', 'Middle East & North Africa',
       'Middle East & North Africa', 'Middle East & North Africa',
       'Middle East & North Africa', 'Middle East & North Africa',
       'Middle East & North Africa', 'Middle East & North Africa',
       'Middle East & North Africa', 'Middle East & North Africa',
       'Middle East & North Africa', 'Middle East & North Africa',
       'Middle East & North Africa', 'Middle East & North Africa',
       'Middle East & North Africa', 'Middle East & North Africa',
       'Middle East & North Africa', 'America', 'America', 'America',
       'America', 'America', 'America', 'America', 'America', 'America',
       'America', 'America', 'America', 'America', 'America', 'America',
       'America', 'America', 'America', 'America', 'America', 'America',
       'America', 'America', 'America', 'America', 'America', 'America',
       'America', 'America', 'America', 'America', 'America', 'America',
       'Europe & Central Asia', 'Europe & Central Asia',
       'Europe & Central Asia', 'Europe & Central Asia',
       'Europe & Central Asia', 'Europe & Central Asia',
       'Europe & Central Asia', 'Europe & Central Asia',
       'Europe & Central Asia', 'Europe & Central Asia',
       'Europe & Central Asia', 'Europe & Central Asia',
       'Europe & Central Asia', 'Europe & Central Asia',
       'Europe & Central Asia', 'Europe & Central Asia',
       'Europe & Central Asia', 'Europe & Central Asia',
       'Europe & Central Asia', 'Europe & Central Asia',
       'Europe & Central Asia', 'Europe & Central Asia',
       'Europe & Central Asia', 'Europe & Central Asia',
       'Europe & Central Asia', 'Europe & Central Asia',
       'Europe & Central Asia', 'Europe & Central Asia',
       'Europe & Central Asia', 'Europe & Central Asia',
       'Europe & Central Asia', 'Europe & Central Asia',
       'Europe & Central Asia', 'Europe & Central Asia',
       'Europe & Central Asia', 'Europe & Central Asia',
       'Europe & Central Asia', 'Europe & Central Asia',
       'Europe & Central Asia', 'Europe & Central Asia',
       'Europe & Central Asia', 'Europe & Central Asia',
       'Europe & Central Asia', 'Europe & Central Asia',
       'Europe & Cent